<h2>This notebook studies the <b>Team Orienteering </b> with Pick-up, Delivery Problem and Transfer (TOPDPT)</h2>

In [16]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [17]:
import os, sys, random, time, pickle
src_dir_ = '/home/tan/Documents/GitHub/pdpt_2022/src'
sys.path.insert(1, src_dir_)

from gurobipy import Model, quicksum, GRB
import numpy as np
from util import generate_node_cargo_size_change, read_pickle, group_cycle_truck, manual_stop
from pathlib import Path
from tvopdpt import tvopdpt_milp_gurobi
dir_ = '/home/tan/Documents/GitHub/pdpt_2022/'
num_ins = 10

In [20]:
def solve_tvopdpt_mip(ins,  # dict contains the data of pdpt instance,
                     gurobi_log_file, # file where all data of pdotw solutions are saved
                     optimize_pdotw_routes = True,
                     max_runtime = 100,
                     verbose = 0):  

    # load data from ins
    selected_truck = ins['truck']
    selected_cargo = ins['cargo']
    selected_node = ins['nodes']
    selected_edge = ins['edge_shortest']    
    # edges = ins['edges']
    # nodes = ins['nodes']
    constant = ins['constant']
    node_cargo_size_change = ins['node_cargo_size_change']
    # path_shortest = ins['path_shortest']
    single_truck_deviation = ins['single_truck_deviation']


    # edge_shortest, path_shortest = replace_edge_by_shortest_length_nx(nodes, edges)
    # single_truck_deviation = calculate_single_truck_deviation(truck, cargo, edge_shortest)
    
    start_time = time.time()
    print(f'========= START [PDOTW with truck ========= ')

    
    created_truck = selected_truck.copy()
    
    # nodes in the cluster
    # Note. cargo['nb_cargo'] = ['size', 'lb_time', 'ub_time','departure_node', 'arrival_node']
    # truck['nb_truck'] = ['departure_node', 'arrival_node', 'max_worktime', 'max_capacity']
    

    if verbose >0:
        print(f'+++ Preprocess data to instantiate a PDOTW MIP')
    if verbose > 2:
        print(f'    [selected_cargo] size: {len(selected_cargo)}')
        for key, value in selected_cargo.items():
            print(f'        {key, value}')
        print(f'    [created_truck] size: {len(created_truck)}')
        for key, value in created_truck.items():
            print(f'       {key, value}')
    
    ### Need to update node_cargo_size_change 
    node_cargo_size_change = generate_node_cargo_size_change(selected_node, selected_cargo)

    ### group cycle and non-cycle trucks
    created_truck_yCycle, created_truck_nCycle, selected_truck = group_cycle_truck(created_truck)  
    
    if verbose > 2:
        print('    [created_truck_yCycle]', created_truck_yCycle)
        print('    [created_truck_nCycle]', created_truck_nCycle)


    if verbose >0:
        print(f'+++ Solve TVOPDPT MILP in Gurobi')
    ### use gurobi to solve the GROW origin PDPTW
    # Note. the pdotw_mip_gurobi function is desgined to take the same arguments as pdpt function
    # but some parameters



    obj_val_MP, runtime_MP, gurobi_res = tvopdpt_milp_gurobi(constant, 
                                                           selected_cargo, 
                                                           selected_truck, 
                                                           created_truck_yCycle, 
                                                           created_truck_nCycle,
                                                           selected_node, 
                                                           selected_edge, 
                                                           node_cargo_size_change, 
                                                           max_runtime, 
                                                           gurobi_log_file, 
                                                           verbose = 1)


    x_sol, z_sol, u_sol, w_sol, S_sol, D_sol, A_sol, Sb_sol, Db_sol, Ab_sol = gurobi_res
    # Index k for truck is pre-defined
    #x_sol: x^k_{ij}, if truck k visit edge (i,j) or not
    #z_sol: z^{kr}_{ij}, if truck k visit edge (i,j) with cargo r
    #u_sol: u^r_i, if cargo r is transfered at node i
    #y_sol: y^k_r, if parcel r is carried by truck k
    #S_sol: x^k_i, total size of cargos on truck k at node i
    #D_sol: D^k_i, depature time of truck k at node i
    #A_sol: A^k_i, arrival time of truck k at node i

    print(f'=== Result: total cargo [{len(selected_cargo.keys())}], MIP obj [{obj_val_MP}]')
    res = {'obj_val_MP': obj_val_MP,
           'runtime_MP': runtime_MP,
           'x_sol':  x_sol,
           'z_sol':  z_sol,
           'u_sol':  u_sol,
           'w_sol':  w_sol,
           'S_sol':  S_sol,
           'D_sol':  D_sol,
           'A_sol':  A_sol,
           'Sb_sol': Sb_sol,
           'Db_sol': Db_sol,
           'Ab_sol': Ab_sol
          }

    return res

In [19]:
def main():
    max_runtime = 20
    case_num, num_trucks = 1, 2

    for ins_idx in range(10):
        pdpt_ins_filename = os.path.join(dir_, f'data/case{case_num}', f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}.pkl')
        print(f'===== START team orienteering pdpt\n      {pdpt_ins_filename}')
        pdpt_ins = read_pickle(pdpt_ins_filename)

        path_ = os.path.join(dir_, f'data/case{case_num}', 'tvopdpt_res')
        Path(path_).mkdir(parents=True, exist_ok=True)

        gurobi_log_file = os.path.join(path_, f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}_gurobi.pkl')

        res = solve_tvopdpt_mip(pdpt_ins,  # dict contains the data of pdpt instance,
                            gurobi_log_file, # file where all data of pdotw solutions are saved
                            optimize_pdotw_routes = True,
                            max_runtime = max_runtime,
                            verbose = 0)

        res_filename = os.path.join(path_, f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}_res.pkl')
        with open(res_filename, 'wb') as f:
            pickle.dump(res, f)

main()

===== START team orienteering pdpt
      /home/tan/Documents/GitHub/pdpt_2022/data/case1/case1_truck2_ins1.pkl
========= START [PDOTW with truck ========= 
Set parameter Heuristics to value 0.5
Set parameter LogFile to value "/home/tan/Documents/GitHub/pdpt_2022/data/case1/tvopdpt_res/case1_truck2_ins1_gurobi.pkl"
9
+++ Gurobi MIP for TVOPDPT [Feasible] 


In [15]:
def check_mip_sol():
    case_num, ins_idx, num_trucks = 1, 0, 2
    pdpt_ins_filename = os.path.join(dir_, f'data/case{case_num}', f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}.pkl')
    print(f'===== START team orienteering pdpt\n      {pdpt_ins_filename}')
    pdpt_ins = read_pickle(pdpt_ins_filename)


    # load data from ins
    selected_truck = pdpt_ins['truck']
    selected_cargo = pdpt_ins['cargo']
    selected_node = pdpt_ins['nodes']
    selected_edge = pdpt_ins['edge_shortest']    
    # edges = ins['edges']
    # nodes = ins['nodes']
    constant = pdpt_ins['constant']
    node_cargo_size_change = pdpt_ins['node_cargo_size_change']
    edge_shortest = pdpt_ins['edge_shortest']
    # path_shortest = ins['path_shortest']
    single_truck_deviation = pdpt_ins['single_truck_deviation']

    print(f'total num. of cargo {len(selected_cargo.keys())}')

    tvopdpt_res_filename = os.path.join(dir_, f'data/case{case_num}', f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}_res.pkl')
    tvopdpt_res = read_pickle(tvopdpt_res_filename)

    print(tvopdpt_res.keys())

    print(sum(tvopdpt_res['u_sol'].values()))
    print(sum(tvopdpt_res['w_sol'].values()))

    print('obj', tvopdpt_res['obj_val_MP'])
    print('runtime_MP', tvopdpt_res['runtime_MP'])
    z_sol = tvopdpt_res['z_sol']

    # truck1, truck2 = selected_truck.keys()
    cargo_delivered = []
    cargo_undelivered = []
    for cargo_key, cargo_value in selected_cargo.items():
        cargo_origin, cargo_dest = cargo_value[-2:]
        print(f'[{cargo_key}], origin [{cargo_origin}], dest [{cargo_dest}]')
        if sum([sum([z_sol[(node, cargo_dest, truck_key, cargo_key)]for node in selected_node if node != cargo_dest]) for truck_key in selected_truck.keys()]) == 1:
            cargo_delivered.append(cargo_key)
            node_curr = cargo_origin
            for node_next in selected_node:
                z_temp = 0
                if node_next!=node_curr:
                    for truck_key in selected_truck.keys():
                        z_temp += z_sol[(cargo_origin, node_next, truck_key, cargo_key)]
                        if z_sol[(cargo_origin, node_next, truck_key, cargo_key)] == 1:
                            print(f'  +++ {cargo_key} from origin [{node_curr}] to node [{node_next}] by Truck {truck_key}')
                            node_curr = node_next
                            
                        if z_temp == 1:
                            break
                if z_temp == 1:
                    break

                    
            while node_curr != cargo_dest:
                z_temp = 0
                for node_next in selected_node:
                    if node_next!=node_curr:
                        for truck_key in selected_truck.keys():
                            z_temp += z_sol[(node_curr, node_next, truck_key, cargo_key)]
                            if z_sol[(node_curr, node_next, truck_key, cargo_key)] == 1:
                                if node_next == cargo_dest:
                                    print(f'  +++ {cargo_key} from node [{node_curr}] to dest [{node_next}] by Truck {truck_key}')
                                else:
                                    print(f'  +++ {cargo_key} from node [{node_curr}] to node [{node_next}] by Truck {truck_key}')
                                node_curr = node_next

                                
                            
                            if z_temp == 1:
                                break
                    if z_temp == 1:
                        break
        else:
            cargo_undelivered.append(cargo_key)

            print(f'[{cargo_key}] is not delivered')

    print(f'{len(cargo_delivered)} cargo delivered: {cargo_delivered}')
    print(f'{len(cargo_undelivered)} cargo undelivered: {cargo_undelivered}')
check_mip_sol()

===== START team orienteering pdpt
      /home/tan/Documents/GitHub/pdpt_2022/data/case1/case1_truck2_ins1.pkl
total num. of cargo 16
dict_keys(['obj_val_MP', 'runtime_MP', 'x_sol', 'z_sol', 'u_sol', 'w_sol', 'S_sol', 'D_sol', 'A_sol', 'Sb_sol', 'Db_sol', 'Ab_sol'])
1
4
obj 14.0
runtime_MP 100.00699710845947
[C44], origin [N5], dest [N4]
[C44] is not delivered
[C89], origin [N10], dest [N15]
  +++ C89 from origin [N10] to node [N15] by Truck T5
[C108], origin [N5], dest [N4]
  +++ C108 from origin [N5] to node [N10] by Truck T5
  +++ C108 from node [N10] to node [N15] by Truck T5
  +++ C108 from node [N15] to dest [N4] by Truck T5
[C128], origin [N10], dest [N15]
  +++ C128 from origin [N10] to node [N15] by Truck T5
[C214], origin [N10], dest [N15]
  +++ C214 from origin [N10] to node [N15] by Truck T5
[C243], origin [N10], dest [N15]
  +++ C243 from origin [N10] to node [N15] by Truck T5
[C1], origin [N7], dest [N22]
  +++ C1 from origin [N7] to node [N27] by Truck T21
  +++ C1 from 

In [ ]:
def postprocess_pdpt_solution(pdpt_ins, tvopdpt_res, verbose = 0):

    selected_cargo = pdpt_ins['cargo']
    selected_truck = pdpt_ins['truck']
    assert len(selected_truck.keys()) == 2
    truck1, truck2 = selected_truck.keys()
    selected_node  = pdpt_ins['nodes']
    selected_edge  = pdpt_ins['edge_shortest']
    
    x_sol = tvopdpt_res['x_sol']
    z_sol = tvopdpt_res['z_sol']
    z_sol = tvopdpt_res['z_sol']
    u_sol = tvopdpt_res['u_sol']
    w_sol = tvopdpt_res['w_sol']

    truck_used = []         # list of trucks used in the solution
    
    # dictionary, for each truck_key, there is a list of cargo carried that was on this truck. E.g., {'T1": ['C1', 'C2', ...]}
    cargo_in_truck = {}   
    for truck_key in selected_truck.keys():
        cargo_in_truck[truck_key] = []

    # dictionary, for each cargo_key, there is a list of truck that carried this cargo. E.g., {'C1": ['T1', 'T2', ...]}
    trucks_per_cargo = {} 
    for cargo_key in selected_cargo.keys():
        trucks_per_cargo[cargo_key] = []

    cargo_delivered = []   # list of delivered cargo
    cargo_undelivered = [] # list of undelivered cargo


    truck_route = {}
    for truck_key in selected_truck.keys():
        truck_route[truck_key] = []
    cargo_route = {}
    for cargo_key in selected_cargo.keys():
        cargo_route[cargo_key] = []

    # dict, for each cargo_key, there is a nested list, each with [transfer_node, truck_from, truck_to]
    cargo_transfer = {}
    for cargo_key in selected_cargo.keys():
        cargo_transfer[cargo_key] = []

    if verbose >0:
        print(f'   +++ Process cargo-to-truck-assignment')
    # Generate cargo_in_truck
    for cargo_key, cargo_value in selected_cargo.items():
        cargo_origin, cargo_dest = cargo_value[-2:]
        # assert sum([z_sol[(cargo_origin, node_, truck1, cargo_key)] for node_ in selected_node if node_!= cargo_origin]) + sum([z_sol[(cargo_origin, node_, truck2, cargo_key)] for node_ in selected_node if node_!= cargo_origin]) == 1
        # assert sum([z_sol[(node_, cargo_dest, truck1, cargo_key)] for node_ in selected_node if node_!= cargo_dest]) +sum([z_sol[(cargo_dest, node_, truck2, cargo_key)] for node_ in selected_node if node_!= cargo_dest]) == 1

        for truck_key in selected_truck.keys():
            if sum([z_sol[(cargo_origin, node_, truck_key, cargo_key)] for node_ in selected_node if node_!= cargo_origin]) + sum([z_sol[(node_, cargo_dest, truck_key, cargo_key)] for node_ in selected_node if node_!= cargo_dest])  >=1:
                cargo_in_truck[truck_key].append(cargo_key)


    if verbose >0:
        print(f'   +++ Process truck_route')
    # postprocess truck_route
    for truck_key in selected_truck:
        source = selected_truck[truck_key][0] # starting from the origin node of each truck
        for node_ in selected_node:
            if node_ != source and x_sol[(source, node_, truck_key)] == 1: # find node_ such that x[source, node_, truck_key] == 1

                if len(truck_route[truck_key]) == 0: # append source as the first node
                        truck_route[truck_key].append(source)
                truck_route[truck_key].append(node_)
                source = node_
                break
        while source != selected_truck[truck_key][1]: # terminate when reach the arival node of each truck
            for node_ in selected_node:
                if node_ != source and x_sol[(source, node_, truck_key)] == 1: # find node_ such that x[source, node_, truck_key] == 1
                    truck_route[truck_key].append(node_)
                    source = node_
                    break

    # RECALL, cargo is a dictionary with the following format:
    # cargo['nb_cargo'] = ['size', 'lb_time', 'ub_time', 'departure_node', 'arrival_node']

    if verbose >0:
        print(f'   +++ Process cargo_route, cargo_delivered and cargo_undelivered')
    for cargo_key, cargo_value in selected_cargo.items():
        cargo_origin, cargo_dest = cargo_value[-2:]
        # if cargo is delivered
        if sum([z_sol[(node_, cargo_dest, truck_key, cargo_key)] for node_ in selected_node for truck_key in selected_truck.keys() if node_!= cargo_dest]) == 1:
            cargo_delivered.append(cargo_key)
            print(f'cargo [{cargo_key}] is delivered')

            #start from cargo_origin
            node_curr = cargo_origin
            #check the next node
            for n_next in selected_node:
                if n_next != node_curr:
                    for truck_key in selected_truck.keys():
                        if z_sol[(node_curr, n_next, truck_key, cargo_key)] == 1:
                            if len(cargo_route[cargo_key]) == 0: # append source as the first node
                                cargo_route[cargo_key].append((truck_key, node_curr))
                            cargo_route[cargo_key].append((truck_key, n_next))
                            node_curr = n_next
                            print(f'cargo_origin [{cargo_origin}], node_next[{n_next}]')
                            break

            while node_curr != cargo_value[-1]:
                for n_next in selected_node:
                    if n_next!=node_curr:
                        for truck_key in selected_truck.keys():
                            if  z_sol[(node_curr, n_next, truck_key, cargo_key)] == 1:
                                cargo_route[cargo_key].append((truck_key, n_next))
                                node_curr = n_next
                                break
 
        else:
            print([ [truck_key, cargo_key, sum(z_sol[(node_, cargo_dest, truck_key, cargo_key)] for node_ in selected_node if node_!= cargo_dest) ] for truck_key in selected_truck.keys()])

            assert sum([z_sol[(node_, cargo_dest, truck_key, cargo_key)] for node_ in selected_node for truck_key in selected_truck.keys() if node_!= cargo_dest]) == 0
            print(f'cargo [{cargo_key}] is undelivered')

            cargo_undelivered.append(cargo_key)

    truck_1, truck_2 = selected_truck.keys()
    if verbose >0:
        print(f'   +++ Process cargo_transfer')
    for cargo_ in selected_cargo.keys():
        for node_curr in selected_node:
            if u_sol[node_, cargo_key] == 1:
                assert sum([z_sol[(node_prev, node_curr, truck_1, cargo_)] for node_prev in selected_node if node_curr != node_prev]) - sum([z_sol[(node_curr, node_next, truck_2, cargo_)] for node_next in selected_node if node_curr != node_next]) == 1
                cargo_transfer[cargo_key].append([node_curr, truck_1, truck_2])
            if w_sol[node_curr, cargo_key] == 1:
                assert sum([z_sol[(node_prev, node_curr, truck_2, cargo_)] for node_prev in selected_node]) - sum([z_sol[(node_curr, node_next, truck_1, cargo_)] for node_next in selected_node]) == 1
                cargo_transfer[cargo_key].append([node_curr, truck_2, truck_1])
                



    processed_res = (truck_used, cargo_delivered, cargo_undelivered, cargo_in_truck, truck_route, cargo_route, cargo_transfer)


    return processed_res


def read_result():
    case_num, ins_idx, num_trucks = 1, 0, 2
    
    pdpt_ins_filename = os.path.join(dir_, f'data/case{case_num}', f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}.pkl')
    pdpt_ins = read_pickle(pdpt_ins_filename)
    
    print('=== summary of pdpt_ins')
    print('\t[', len(pdpt_ins['cargo'].keys()),'] cargos, [', len(pdpt_ins['truck'].keys()),'] trucks')
    res_filename = os.path.join(dir_, f'data/case{case_num}', f'case{case_num}_truck{num_trucks}_ins{ins_idx+1}_res.pkl')
    tvopdpt_res = read_pickle(res_filename, verbose = 0) 
    print('=== Start Postprocess mip sol')
    processed_res  = postprocess_pdpt_solution(pdpt_ins, tvopdpt_res, verbose = 1)
    
    truck_used, cargo_delivered, cargo_undelivered, trucks_per_cargo, cargo_in_truck, truck_route, cargo_route, cargo_transfer = processed_res

    print(f'+++ list of cargos sucessfully delivered \n{cargo_delivered}')
          
    print(cargo_undelivered)
    
read_result()